In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from itertools import cycle
from statistics import mean 
import random
from random import randint
import numpy as np
import time

In [7]:
symbols = []
companyNames = []
companySectors = []
userAgentList = []

useragents = open("useragents.txt", "r")

for line in useragents:
    userAgentList.append(line.replace('\n', ''))
    

delays = []

for x in range(100):
    delays.append(randint(50, 250) / 100)

In [8]:
def getProxies(inURL):
    
    page = requests.get(inURL)
    soup = BeautifulSoup(page.text, 'html.parser')
    terms = soup.find_all('tr')
    IPs = []

    for x in range(len(terms)):  
        
        term = str(terms[x])        
        
        if '<tr><td>' in str(terms[x]):
            pos1 = term.find('d>') + 2
            pos2 = term.find('</td>')

            pos3 = term.find('</td><td>') + 9
            pos4 = term.find('</td><td>US<')
            
            IP = term[pos1:pos2]
            port = term[pos3:pos4]
            
            if '.' in IP and len(port) < 6:
                IPs.append(IP + ":" + port)
                #print(IP + ":" + port)

    return IPs 


proxyURL = "https://www.us-proxy.org/"
pxs = getProxies(proxyURL)
proxyPool = cycle(pxs)

In [9]:
def clean(stng):
    
    output = ''

    for letter in stng:
        if letter != '[' and letter != ']' and letter != "'": #and letter != ' ':
            output += letter
        
    return output


def cleanName(stng):
    
    output = stng.lower()
    
    stopwords = [' corporation', ' inc.', ' company', ' plc', '.com', ' co.', ' l.p.', ' ltd', ' corp.', ' limited' ' &amp;', ' ltd.']
    
    for word in stopwords:
        output = output.replace(word, '')
    
    output = output.replace(' ', '-')
    
    return output


def appendGrowth(lst):
    
    output = []
    
    for x in range(len(lst) - 1):
        
        try:
            diff = lst[x] - lst[x + 1]
            
            if diff > 0:
                output.append(round((lst[x] - lst[x + 1]) / abs(lst[x + 1]) * 100, 2))
            else:
                output.append(round((lst[x] - lst[x + 1]) / lst[x + 1] * 100, 2))
                
        except:
            continue
            
    return output



def getAvg(lst):
    
    summ = 0
    nums = 0
    
    for num in lst:
        if num != 0.0:
            summ += num
            nums += 1
    
    try:
        return round(summ / nums, 2)
    except:
        return 0 
    
    
    
def getSymbolsCSV(fileName):
    
    global symbols
    global companyNames
    
    print('\nLoading data from ' + fileName)

    with open(fileName) as csvfile:
    
        readCSV = csv.reader(csvfile, delimiter=',')
        
        for row in readCSV:
            
            if row[0] =='ï»¿A':
                symbols.append('A')
            else:
                symbols.append(row[0])
                
            companyNames.append(cleanName(row[1]))
            companySectors.append(row[2])
                

def exportData(fileName):

    MyFile = open(fileName, 'w')

    header = 'Symbol, Company Name, Sector, Score, AvgYearlyRevGrowth, NetGrowth, AvgGrossMarginGrowth, AvgNetIncGrowth, AvgNetIncMarginGrowth' + '\n'

    MyFile.write(header)

    for row in overall:
        MyFile.write(clean(str(row)))
        MyFile.write('\n')

    MyFile.close()

    print('\nSaved as ' + fileName)

In [10]:
#fileName = 'C:\\Users\\faiza\\Automated-Fundamental-Analysis\\SymbolData\\S&P500Symbols.csv'
fileName = 'SymbolData\\SmallCapSymbols.csv'
getSymbolsCSV(fileName)


Loading data from SymbolData\SmallCapSymbols.csv


In [11]:
overall = []

def getIncomeStatementScore(sym, ind):
    
    url = 'https://www.macrotrends.net/stocks/charts/' + sym + '/' + companyNames[ind] + '/financial-statements'
    print(url)
    
    agent = random.choice(userAgentList)
    headers = {'User-Agent': agent}
    pageError = False
    
    yearlyRevenues = []
    yearlyCogs = []
    yearlyNetIncome = []
    
    
    try:
        page = requests.get(url, headers=headers, proxies = {"http": next(proxyPool)})
        soup = str(BeautifulSoup(page.text, 'html.parser'))
    except:
        pageError = True
    
    if pageError == False:
    
        table = soup[soup.find('var originalData') : soup.find('localdata: originalData,')]
        table = table.split('","')


        startCogs = False
        endCogs = False
        startNetIncome = False
        endNetIncome = False


        for line in table:

            num = line[line.find('":"') + 3 : ]
            appended = False


            if '"field_name":"' in line:
                num = line[line.find(':"') + 2 : line.find('"}')]


            validCheck = len(num) != 0 and len(num) < 60 and num != '-'


            if startCogs == False and validCheck == True and appended == False:

                try:
                    yearlyRevenues.append(float(num))
                    appended = True
                except:
                    continue

            if 'cost-goods-sold' in line:
                startCogs = True


            if endCogs == False and startCogs == True and validCheck == True and appended == False:

                try:
                    yearlyCogs.append(float(num))
                    appended = True
                except:
                    continue


            if 'gross-profit' in line:
                endCogs = True

            if 'net-income' in line:
                startNetIncome = True

            if endNetIncome == False and startNetIncome == True and validCheck == True and appended == False:

                try:
                    yearlyNetIncome.append(float(num))
                    appended = True
                except:
                    continue

            if 'ebitda' in line:
                endNetIncome = True


        yearlyRevenues = yearlyRevenues[:4]
        yearlyCogs = yearlyCogs[:4]
        yearlyNetIncome = yearlyNetIncome[:4]

    
    if len(yearlyCogs) > 0 and len(yearlyRevenues) == len(yearlyCogs):
        
        print(sym + '\n')
        print('Yearly Revenues:', yearlyRevenues)
        print('Yearly Cogs:', yearlyCogs)
        print('Yearly Net Income:', yearlyNetIncome)
        print()


        grossMargins = []
        netMargins = []

        
        try:
            
            for x in range(len(yearlyRevenues)):
                grossMargins.append(round(((yearlyRevenues[x] - yearlyCogs[x]) / yearlyRevenues[x] * 100), 2))

            for x in range(len(yearlyRevenues)):
                netMargins.append(round((yearlyNetIncome[x] / yearlyRevenues[x] * 100), 2))



            yearlyRevGrowth = appendGrowth(yearlyRevenues)
            yearlyCogsGrowth = appendGrowth(yearlyCogs)
            yearlyNetIncGrowth = appendGrowth(yearlyNetIncome)
            yearlyGrossMarginGrowth = appendGrowth(grossMargins)
            yearlyNetIncMarginGrowth = appendGrowth(netMargins)

            avgYearlyRevGrowth = getAvg(yearlyRevGrowth)
            avgYearlyCogsGrowth = getAvg(yearlyCogsGrowth)
            avgNetIncGrowth = getAvg(yearlyNetIncGrowth)
            avgGrossMarginGrowth = getAvg(yearlyGrossMarginGrowth)
            avgNetIncMarginGrowth = getAvg(yearlyNetIncMarginGrowth)

            netGrowth = round(avgYearlyRevGrowth - avgYearlyCogsGrowth, 2)


            print('Yearly Rev Growth:', yearlyRevGrowth, avgYearlyRevGrowth)
            print('Yearly Cogs Growth:', yearlyCogsGrowth, avgYearlyCogsGrowth)
            print('Yearly Net Inc Growth:', yearlyNetIncGrowth, avgNetIncGrowth)

            print('\nGross Margins:', grossMargins, getAvg(grossMargins))
            print('Yearly Gross Margin Growth:', yearlyGrossMarginGrowth, avgGrossMarginGrowth)

            print('\nNet Margins:', netMargins, getAvg(netMargins))
            print('Yearly Net Margin Growth:', yearlyNetIncMarginGrowth, avgNetIncMarginGrowth)

            print('\nNet Growth', netGrowth)

            score = round(((avgYearlyRevGrowth * 20) + (netGrowth * 20) + (avgGrossMarginGrowth * 10) + (avgNetIncGrowth + avgNetIncMarginGrowth) / 10), 2)

            overall.append([sym, companyNames[ind], companySectors[ind], score, avgYearlyRevGrowth, netGrowth, avgGrossMarginGrowth, avgNetIncGrowth, avgNetIncMarginGrowth])


            print('\nFinal Score:', score)
            print()
            
        except:
            print('Err:', sym)

In [12]:
for i, stock in enumerate(symbols):
    getIncomeStatementScore(stock, i)
        
exportData('SmallCapIncStatementRatings2.csv')

https://www.macrotrends.net/stocks/charts/AA/alcoa/financial-statements
AA

Yearly Revenues: [10433.0, 13403.0, 11652.0, 9318.0]
Yearly Cogs: [8537.0, 10053.0, 8950.0, 7877.0]
Yearly Net Income: [-1125.0, 250.0, 279.0, -400.0]

Yearly Rev Growth: [-22.16, 15.03, 25.05] 5.97
Yearly Cogs Growth: [-15.08, 12.32, 13.62] 3.62
Yearly Net Inc Growth: [-550.0, -10.39, 169.75] -130.21

Gross Margins: [18.17, 24.99, 23.19, 15.46] 20.45
Yearly Gross Margin Growth: [-27.29, 7.76, 50.0] 10.16

Net Margins: [-10.78, 1.87, 2.39, -4.29] -2.7
Yearly Net Margin Growth: [-676.47, -21.76, 155.71] -180.84

Net Growth 2.35

Final Score: 236.9

https://www.macrotrends.net/stocks/charts/AAL/american-airlines-group/financial-statements
AAL

Yearly Revenues: [45768.0, 44541.0, 42622.0, 40142.0]
Yearly Cogs: [33397.0, 32651.0, 29590.0, 26891.0]
Yearly Net Income: [1686.0, 1412.0, 1282.0, 2584.0]

Yearly Rev Growth: [2.75, 4.5, 6.18] 4.48
Yearly Cogs Growth: [2.28, 10.34, 10.04] 7.55
Yearly Net Inc Growth: [19.41

In [180]:
exportData('SmallCapIncStatementRatings2.csv')


Saved as SmallCapIncStatementRatings.csv


In [ ]:
sym = 'AAPL'

url = 'https://www.macrotrends.net/stocks/charts/' + sym + '/zedge/financial-statements'

agent = random.choice(userAgentList)
headers = {'User-Agent': agent}

page = requests.get(url, headers=headers, proxies = {"http": next(proxyPool)})
soup = str(BeautifulSoup(page.text, 'html.parser'))

In [ ]:
table = soup[soup.find('var originalData') : soup.find('localdata: originalData,')]
table = table.split('","')

In [ ]:
yearlyRevenues = []
yearlyCogs = []
yearlyNetIncome = []


startCogs = False
endCogs = False
startNetIncome = False
endNetIncome = False
    

for line in table:
    
    num = line[line.find('":"') + 3 : ]
    appended = False
    
    
    if '"field_name":"' in line:
        num = line[line.find(':"') + 2 : line.find('"}')]

    
    validCheck = len(num) != 0 and len(num) < 60 and num != '-'
    
    
    if startCogs == False and validCheck == True and appended == False:
        
        try:
            yearlyRevenues.append(float(num))
            appended = True
        except:
            continue
    
    if 'cost-goods-sold' in line:
        startCogs = True
        
        
    if endCogs == False and startCogs == True and validCheck == True and appended == False:
        
        try:
            yearlyCogs.append(float(num))
            appended = True
        except:
            continue
            
            
    if 'gross-profit' in line:
        endCogs = True
        
    if 'net-income' in line:
        startNetIncome = True
        
    if endNetIncome == False and startNetIncome == True and validCheck == True and appended == False:
        
        try:
            yearlyNetIncome.append(float(num))
            appended = True
        except:
            continue
    
    if 'ebitda' in line:
        endNetIncome = True
    

yearlyRevenues = yearlyRevenues[:4]
yearlyCogs = yearlyCogs[:4]
yearlyNetIncome = yearlyNetIncome[:4]

print(yearlyRevenues)
print(yearlyCogs)
print(yearlyNetIncome)